In [1]:
import torch

torch.cuda.is_available()

True

In [28]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple/


Exception ignored in: <function _releaseLock at 0x7fdec7105550>
Traceback (most recent call last):
  File "/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


In [3]:
!pip install peft

In [4]:
!pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [2]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

from peft import PeftModel
import faiss
import pickle

In [3]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()


In [7]:
!nvidia-smi

Wed Oct 30 21:12:33 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX 6000...  Off  | 00000000:17:00.0 Off |                  Off |
| 30%   37C    P8    26W / 300W |  12922MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        breakpoint()
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-base"):
    """
    Faiss DB: 
    Embedding 작업(인코딩을 통해 vector들을 하나의 vector space에 투영)이 끝난 뒤,
    vector space 내의 벡터끼리 유사도 기반 검색 서비스를 더 빠르게 지원
    벡터의 유사도 측정, 클러스터링에 효율적인 라이브러리

    전통적인 코사인 유사도 < Faiss DB(gpu 지원)
    """
    
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db




def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    pdf_files = [file for file in os.listdir(base_directory) if file.endswith('.pdf')]

    
    for path in tqdm(pdf_files, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        
        # 빈 chunks 처리
        if not chunks:
            print(f"Skipping {pdf_title} due to empty content.")
            continue
        
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr", 
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        

        faiss_directory = "./faiss/" + base_directory.split('/')[-1] + "/"
        faiss.write_index(db.index, faiss_directory + pdf_title + "_faiss_db.index")

        # 필요한 매핑 객체를 추출하여 저장
        with open(faiss_directory + pdf_title + "_index_to_docstore_id.pkl", "wb") as f:
            pickle.dump(db.index_to_docstore_id, f)

        with open(faiss_directory + pdf_title + "_docstore.pkl", "wb") as f:
            pickle.dump(db.docstore, f)
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases



In [9]:
fish_data_directory = './data/fish' # Your Base Directory
# df = pd.read_csv(data_directory + '/test.csv')
fish_pdf_databases = process_pdfs_from_dataframe(fish_data_directory)

Processing PDFs:   0%|          | 0/11 [00:00<?, ?it/s]

Processing 숭어...


/tmp/ipykernel_797068/3492978855.py:35: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
Processing PDFs:   9%|▉         | 1/11 [00:06<01:01,  6.17s/it]

Processing 돔류...


Processing PDFs:  18%|█▊        | 2/11 [00:10<00:47,  5.33s/it]

...

Processing PDFs:  82%|████████▏ | 9/11 [00:42<00:09,  4.61s/it]

Processing 강도다리...


Processing PDFs:  91%|█████████ | 10/11 [00:46<00:04,  4.45s/it]

Processing 뱀장어...


Processing PDFs: 100%|██████████| 11/11 [00:52<00:00,  4.80s/it]


In [10]:
shellfish_data_directory = './data/shellfish' # Your Base Directory
# df = pd.read_csv(data_directory + '/test.csv')
shellfish_pdf_databases = process_pdfs_from_dataframe(shellfish_data_directory)
shellfish_pdf_databases

Processing PDFs:   0%|          | 0/3 [00:00<?, ?it/s]

Processing 가리비...


Processing PDFs:  33%|███▎      | 1/3 [00:04<00:09,  4.98s/it]

Processing 전복...


Processing PDFs:  67%|██████▋   | 2/3 [00:08<00:04,  4.28s/it]

Processing 참굴...


Processing PDFs: 100%|██████████| 3/3 [00:13<00:00,  4.33s/it]


{'가리비': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4b3e9d9100>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4b3e9d9100>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})},
 '전복': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4b3e699880>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4b3e699880>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})},
 '참굴': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4add122070>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4add122070>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})}}

In [11]:
seaweed_data_directory = './data/seaweed' # Your Base Directory
# df = pd.read_csv(data_directory + '/test.csv')
seaweed_pdf_databases = process_pdfs_from_dataframe(seaweed_data_directory)
seaweed_pdf_databases

Processing PDFs:   0%|          | 0/6 [00:00<?, ?it/s]

Processing 미역...


Processing PDFs:  17%|█▋        | 1/6 [00:04<00:21,  4.36s/it]

Processing 김...


Processing PDFs:  33%|███▎      | 2/6 [00:10<00:21,  5.28s/it]

...

Processing 청각...


Processing PDFs:  83%|████████▎ | 5/6 [00:22<00:04,  4.25s/it]

Processing 넓미역...


Processing PDFs: 100%|██████████| 6/6 [00:25<00:00,  4.32s/it]


{'미역': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4b3e85e100>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4b3e85e100>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})},
 '김': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4c6d6530a0>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4c6d6530a0>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})},
 '곰피': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4adcc0d130>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4adcc0d130>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})},
 '모자반': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4b3e5b8070>,
  'retriever': Ve

In [12]:
etc_data_directory = './data/etc' # Your Base Directory
# df = pd.read_csv(data_directory + '/test.csv')
etc_pdf_databases = process_pdfs_from_dataframe(etc_data_directory)
etc_pdf_databases

Processing PDFs:   0%|          | 0/4 [00:00<?, ?it/s]

Processing 해삼...


Processing PDFs:  25%|██▌       | 1/4 [00:03<00:09,  3.29s/it]

Processing 큰징거미새우...


Processing PDFs:  50%|█████     | 2/4 [00:06<00:06,  3.32s/it]

Processing 멍게...


Processing PDFs:  75%|███████▌  | 3/4 [00:10<00:03,  3.56s/it]

Processing 흰다리새우...


Processing PDFs: 100%|██████████| 4/4 [00:15<00:00,  3.94s/it]


{'해삼': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4b3e3f21f0>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4b3e3f21f0>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})},
 '큰징거미새우': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4b3e36f0d0>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4b3e36f0d0>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})},
 '멍게': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4adcc080d0>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4adcc080d0>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})},
 '흰다리새우': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4adc091130>,
  'retriev

In [13]:
fish_pdf_databases

{'숭어': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4b4dff4610>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4b4dff4610>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})},
 '돔류': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4b4dff4fd0>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4b4dff4fd0>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})},
 '조피볼락': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4b4838a160>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f4b4838a160>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})},
 '비단잉어': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f4b1a309160>,
  'retriever'

In [34]:
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID 
    model_id = "rtzr/ko-gemma-2-9b-it"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False
    

    # 모델 로드 및 양자화 설정 적용
    model = Gemma2ForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True )

#     model = PeftModel.from_pretrained(model, "./persona/checkpoint-200",is_trainable=True)

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=450,
    )

    # hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    # return hf

    return text_generation_pipeline


In [5]:
# LLM 파이프라인
llm = setup_llm_pipeline()

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_825000/2738076374.py:37: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  hf = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
df_question = pd.read_csv("data/question.csv")
df_question.loc[-1] = df_question.columns  # 기존 컬럼 이름을 첫 번째 행으로 추가
df_question.index = df_question.index + 1  # 인덱스를 1씩 증가
df_question = df_question.sort_index()     # 인덱스를 기준으로 정렬하여 첫 번째 행으로 만듦

# 새로운 컬럼 이름을 지정
df_question.columns = ['species', 'question', 'mm','answer']  # 원하는 컬럼명으로 지정
df_question = df_question.drop(columns=['mm'])

# 새로운 컬럼 'new_column'을 추가하고 인덱스 범위에 따라 다르게 값 할당
df_question['category'] = None  # 먼저 컬럼을 초기화

df_question.loc[0:33, 'category'] = 'fish'
df_question.loc[33:42, 'category'] = 'shellfish'
df_question.loc[42:60, 'category'] = 'seaweed'
df_question.loc[60:, 'category'] = 'etc'

df_question.to_csv("data/question.csv")

ValueError: Length mismatch: Expected axis has 5 elements, new values have 4 elements

In [ ]:
df_question = pd.read_csv("data/question.csv")
df_question = df_question.drop(columns=['Unnamed: 0'])

df_question

,species,question,answer,category
0,넙치,형태적 특징이 뭐야?,몸은 긴 타원형이고 입은 넙치속의 다른 어종에 비하여 대단히 크다. 윗턱의 끝은 눈...,fish
1,넙치,사육 수온이 몇 도야?,21~24℃,fish
2,넙치,자어일 때 먹이가 뭐야?,"식물플랑크톤, 동물플랑크톤(로티퍼, 알테미아)",fish
3,조피볼락,형태적 특징이 뭐야?,체색은 암회갈색으로 복부는 연한 빛을 나타내고 몸 옆에는 분명치 않은 흑갈색의 가로...,fish
4,조피볼락,양식 수온이 뭐야?,15~18°C,fish
...,...,...,...,...
67,멍게,적정 수온이 몇 도야?,10~16°C,etc
68,멍게,유생 때 먹는 먹이가 뭐야?,"식물성 플랑크톤, 해산 클로렐라",etc
69,흰다리새우,형태적 특징이 뭐야?,형태 및 색깔은 대하와 매우 유사하여 구분이 다소 어렵지만 이마뿔(액각)이 대하에 ...,etc
70,흰다리새우,성장 적수온이 몇 도야?,23~30°C,etc


In [36]:
from llm import *


# 임베딩 설정
model_path = "intfloat/multilingual-e5-base"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_path,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 모델 및 토크나이저 로드 (한 번만 실행)
model_id = "rtzr/ko-gemma-2-9b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Gemma2ForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)



# 파라미터만 변경하여 파이프라인 재생성 함수
def create_pipeline_with_params(temperature=0.7, top_p=0.9, max_new_tokens=450):
    return pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=temperature,
        top_p=top_p,
        max_new_tokens=max_new_tokens,
        return_full_text=False
    )


def test_rag(question, category, species, k, fetch_k, temperature, top_p):
    faiss_db_directory = "./faiss/" + category + "/" + species

    # 빈 docstore와 index_to_docstore_id 생성
    with open(faiss_db_directory + "_index_to_docstore_id.pkl", "rb") as f:
        index_to_docstore_id = pickle.load(f)

    with open(faiss_db_directory + "_docstore.pkl", "rb") as f:
        docstore = pickle.load(f)

    # 인덱스 로드 및 FAISS 초기화
    index = faiss.read_index(faiss_db_directory + "_faiss_db.index")
    db = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id
    )

    text_generation_pipeline = create_pipeline_with_params(temperature, top_p)
    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': k, 'fetch_k': fetch_k})
    rag_chain = rag(retriever, hf)

    full_response = rag_chain.invoke(question)
    
    return full_response



# test_row = df_question.iloc[0]  # 첫 번째 행 선택

# # 함수 실행 결과 확인
# test_result = test_rag(test_row['question'], test_row['category'], test_row['species'])
# print("Test Result:", test_result)


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
# 각 컬럼별 null 값 개수 확인
null_counts = df_question.isna().sum()
print(null_counts)


species     0
question    0
answer      0
category    0
dtype: int64


In [14]:
!pip install dask

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.7 MB/s eta 0:00:00
Using cached click-8.1.7-py3-none-any.whl (97 kB)


In [16]:
# apply를 사용해 결과를 임시로 저장
test_results = df_question.apply(lambda row: test_rag(row['question'], row['category'], row['species']), axis=1)

# 결과 확인
print(test_results.head())


0    넙치의 몸은 긴 타원형이고 입은 넙치속의 다른 어종에 비하여 대단히 크다. 윗턱의 ...
1                              사육 수온은 15∼22℃이다. \n\n\n
2                   자어는 로티퍼나 알테미아 유생을 먹이로 섭취한다. \n\n\n
3    조피볼락의 형태적 특징은 체색이 암회갈색으로 복부는 연한 빛을 나타내고 몸 옆에는 ...
4    양식 수온은 생명현상의 71초는 생체내의 화학적 변화라 할 수 있으며，01들 화학적...
dtype: object


In [ ]:
df_question['pred'] = test_results
df_question.to_csv('data/question.csv')

In [26]:
# 질문에 대한 답을 기록할 새로운 컬럼 생성
from tqdm import tqdm

# tqdm을 적용할 수 있도록 설정
tqdm.pandas()

# progress_apply로 tqdm을 적용하여 진행 상황 표시
df_question['pred'] = df_question.progress_apply(lambda row: test_rag(row['question'], row['category'], row['species']), axis=1)

,species,question,answer,category,pred
0,넙치,형태적 특징이 뭐야?,몸은 긴 타원형이고 입은 넙치속의 다른 어종에 비하여 대단히 크다. 윗턱의 끝은 눈...,fish,yntaxException
1,넙치,사육 수온이 몇 도야?,21~24℃,fish,ftagPool
2,넙치,자어일 때 먹이가 뭐야?,"식물플랑크톤, 동물플랑크톤(로티퍼, 알테미아)",fish,esternos
3,조피볼락,형태적 특징이 뭐야?,체색은 암회갈색으로 복부는 연한 빛을 나타내고 몸 옆에는 분명치 않은 흑갈색의 가로...,fish,\n\n
4,조피볼락,양식 수온이 뭐야?,15~18°C,fish,ftagPool
...,...,...,...,...,...
67,멍게,적정 수온이 몇 도야?,10~16°C,etc,\n
68,멍게,유생 때 먹는 먹이가 뭐야?,"식물성 플랑크톤, 해산 클로렐라",etc,ContentAsync
69,흰다리새우,형태적 특징이 뭐야?,형태 및 색깔은 대하와 매우 유사하여 구분이 다소 어렵지만 이마뿔(액각)이 대하에 ...,etc,ftagPool
70,흰다리새우,성장 적수온이 몇 도야?,23~30°C,etc,ftagPool


In [23]:
df_selected = df_question[['answer', 'pred']]
df_selected

,answer,pred
0,몸은 긴 타원형이고 입은 넙치속의 다른 어종에 비하여 대단히 크다. 윗턱의 끝은 눈...,넙치의 몸은 긴 타원형이고 입은 넙치속의 다른 어종에 비하여 대단히 크다. 윗턱의 ...
1,21~24℃,사육 수온은 15∼22℃이다. \n\n\n
2,"식물플랑크톤, 동물플랑크톤(로티퍼, 알테미아)",자어는 로티퍼나 알테미아 유생을 먹이로 섭취한다. \n\n\n
3,체색은 암회갈색으로 복부는 연한 빛을 나타내고 몸 옆에는 분명치 않은 흑갈색의 가로...,조피볼락의 형태적 특징은 체색이 암회갈색으로 복부는 연한 빛을 나타내고 몸 옆에는 ...
4,15~18°C,양식 수온은 생명현상의 71초는 생체내의 화학적 변화라 할 수 있으며，01들 화학적...
...,...,...
67,10~16°C,적정 수온은 5~24도 입니다. \n\n\n
68,"식물성 플랑크톤, 해산 클로렐라","유생은 규조류, 원생동물, 갑각류 중 요각류, 연체류의 유생, 각종 양식생물의 배설..."
69,형태 및 색깔은 대하와 매우 유사하여 구분이 다소 어렵지만 이마뿔(액각)이 대하에 ...,흰다리새우의 형태적 특징은 흰다리새우의 생물학적 특성 부분에서 자세히 설명되어 있습...
70,23~30°C,성장 적수온은 25~30도입니다.\n\n\n\n\n


In [27]:
!pip install nltk
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=437e313df62918df5a6ed5a8a5044239edc488844e326ee6625239927338335a
  Stored in directory: /home/jiyoon/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [28]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

# nltk 다운로드 (처음 실행 시 필요)
nltk.download('punkt')

# BLEU 점수 계산
df_selected['BLEU'] = df_selected.apply(
    lambda row: sentence_bleu([row['answer'].split()], row['pred'].split()), axis=1
)

# BLEU 점수 평균
bleu_avg = df_selected['BLEU'].mean()
print("Average BLEU score:", bleu_avg)


Average BLEU score: 0.05908808425151345


[nltk_data] Downloading package punkt to /home/jiyoon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 co

In [29]:
from rouge_score import rouge_scorer

# ROUGE 점수 계산
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
df_selected['ROUGE'] = df_selected.apply(
    lambda row: scorer.score(row['answer'], row['pred']), axis=1
)

# ROUGE1, ROUGE2, ROUGE-L의 평균 계산
rouge1_avg = df_selected['ROUGE'].apply(lambda x: x['rouge1'].fmeasure).mean()
rouge2_avg = df_selected['ROUGE'].apply(lambda x: x['rouge2'].fmeasure).mean()
rougeL_avg = df_selected['ROUGE'].apply(lambda x: x['rougeL'].fmeasure).mean()

print("Average ROUGE-1 score:", rouge1_avg)
print("Average ROUGE-2 score:", rouge2_avg)
print("Average ROUGE-L score:", rougeL_avg)


Average ROUGE-1 score: 0.13872431127333085
Average ROUGE-2 score: 0.08516589506172839
Average ROUGE-L score: 0.13790731780927856


/tmp/ipykernel_825000/3332143785.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['ROUGE'] = df_selected.apply(


In [33]:
!pip install optuna

In [37]:
import optuna
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# 전체 질문에 대해 평가를 수행하는 Optuna 목표 함수
def objective(trial):
    # 파라미터 샘플링
    k = trial.suggest_int("k", 3, 10)
    fetch_k = trial.suggest_int("fetch_k", 5, 15)
    temperature = trial.suggest_float("temperature", 0.1, 1.0)
    top_p = trial.suggest_float("top_p", 0.7, 1.0)
    
    # 성능 점수를 저장할 리스트
    bleu_scores = []
    rouge_scores = []

    # df_question의 각 질문에 대해 test_rag 함수 실행
    for _, row in df_question.iterrows():
        test_result = test_rag(
            question=row['question'],
            category=row['category'],
            species=row['species'],
            k=k,
            fetch_k=fetch_k,
            temperature=temperature,
            top_p=top_p
        )
        
        # BLEU 및 ROUGE 점수 계산
        bleu_score = sentence_bleu([row['answer'].split()], test_result.split())
        scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
        rouge_score = scorer.score(row['answer'], test_result)['rouge1'].fmeasure
        
        # 점수 리스트에 추가
        bleu_scores.append(bleu_score)
        rouge_scores.append(rouge_score)

    # BLEU와 ROUGE 점수의 평균 계산
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge = sum(rouge_scores) / len(rouge_scores)
    
    # 최적화 대상 - BLEU와 ROUGE의 평균 합
    return avg_bleu + avg_rouge

# Optuna 튜닝 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# 최적 파라미터
best_params = study.best_params
print("Best parameters:", best_params)


[I 2024-11-03 15:48:36,115] A new study created in memory with name: no-name-e67e2ace-1a0f-4e56-8559-2bec9e5a8f3a
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7104261148921122` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9406248330693397` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperatur

Best parameters: {'k': 10, 'fetch_k': 11, 'temperature': 0.1158286270064956, 'top_p': 0.7950143640975952}


## 파라미터 테스트

In [38]:
from llm import *


# 임베딩 설정
model_path = "intfloat/multilingual-e5-base"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_path,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 모델 및 토크나이저 로드 (한 번만 실행)
model_id = "rtzr/ko-gemma-2-9b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Gemma2ForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)



# 파라미터만 변경하여 파이프라인 재생성 함수
def create_pipeline_with_params(temperature=0.1158286270064956, top_p=0.7950143640975952, max_new_tokens=450):
    return pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=temperature,
        top_p=top_p,
        max_new_tokens=max_new_tokens,
        return_full_text=False
    )

text_generation_pipeline = create_pipeline_with_params()
hf = HuggingFacePipeline(pipeline=text_generation_pipeline)


def test_rag(question, category, species):
    faiss_db_directory = "./faiss/" + category + "/" + species

    # 빈 docstore와 index_to_docstore_id 생성
    with open(faiss_db_directory + "_index_to_docstore_id.pkl", "rb") as f:
        index_to_docstore_id = pickle.load(f)

    with open(faiss_db_directory + "_docstore.pkl", "rb") as f:
        docstore = pickle.load(f)

    # 인덱스 로드 및 FAISS 초기화
    index = faiss.read_index(faiss_db_directory + "_faiss_db.index")
    db = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id
    )

    retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 10, 'fetch_k': 11})
    rag_chain = rag(retriever, hf)

    full_response = rag_chain.invoke(question)
    
    return full_response



test_row = df_question.iloc[0]  # 첫 번째 행 선택

# 함수 실행 결과 확인
test_result = test_rag(test_row['question'], test_row['category'], test_row['species'])
print("Test Result:", test_result)


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1158286270064956` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7950143640975952` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Test Result: 넙치는 긴 타원형이고 입은 넙치속의 다른 어종에 비하여 대단히 크다. 윗턱의 끝은 눈보다 후방에 있고 양턱의 이빨은 강하다. 유안측의 비늘은 빗비늘이고, 무안측의 비늘은 둥근비늘이다. 





In [39]:
# apply를 사용해 결과를 임시로 저장
optimized_test_results = df_question.apply(lambda row: test_rag(row['question'], row['category'], row['species']), axis=1)
df_question['optimized_pred'] = optimized_test_results

/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1158286270064956` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7950143640975952` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1158286270064956` -- this flag is only used in sample-based generation modes. You should set `do_s

In [ ]:
df_question.to_csv("data/question.csv")

In [42]:
df_question

,species,question,answer,category,pred,optimized_pred
0,넙치,형태적 특징이 뭐야?,몸은 긴 타원형이고 입은 넙치속의 다른 어종에 비하여 대단히 크다. 윗턱의 끝은 눈...,fish,넙치의 몸은 긴 타원형이고 입은 넙치속의 다른 어종에 비하여 대단히 크다. 윗턱의 ...,넙치는 긴 타원형이고 입은 넙치속의 다른 어종에 비하여 대단히 크다. 윗턱의 끝은 ...
1,넙치,사육 수온이 몇 도야?,21~24℃,fish,사육 수온은 15∼22℃이다. \n\n\n,"사육 수온은 21∼24℃가 적정하며, 사료효율이 가장 높은 수온은 18∼24℃이다...."
2,넙치,자어일 때 먹이가 뭐야?,"식물플랑크톤, 동물플랑크톤(로티퍼, 알테미아)",fish,자어는 로티퍼나 알테미아 유생을 먹이로 섭취한다. \n\n\n,넙치 자어는 로티퍼와 알테미아를 먹이로 섭취한다. \n\n\n
3,조피볼락,형태적 특징이 뭐야?,체색은 암회갈색으로 복부는 연한 빛을 나타내고 몸 옆에는 분명치 않은 흑갈색의 가로...,fish,조피볼락의 형태적 특징은 체색이 암회갈색으로 복부는 연한 빛을 나타내고 몸 옆에는 ...,조피볼락의 형태적 특징은 다음과 같습니다. 산출직후에는 크기가 6.25±0.08mm...
4,조피볼락,양식 수온이 뭐야?,15~18°C,fish,양식 수온은 생명현상의 71초는 생체내의 화학적 변화라 할 수 있으며，01들 화학적...,"양식 수온은 조피볼락의 경우 15~20°C에서 가장 좋으며, 23°C 이상이 되면 ..."
...,...,...,...,...,...,...
67,멍게,적정 수온이 몇 도야?,10~16°C,etc,적정 수온은 5~24도 입니다. \n\n\n,멍게의 적정 수온은 10~16도 입니다. \n\n\n
68,멍게,유생 때 먹는 먹이가 뭐야?,"식물성 플랑크톤, 해산 클로렐라",etc,"유생은 규조류, 원생동물, 갑각류 중 요각류, 연체류의 유생, 각종 양식생물의 배설...",멍게 유생은 부착기에 부착하기 전까지 먹이를 먹지 않습니다. \n\n\n
69,흰다리새우,형태적 특징이 뭐야?,형태 및 색깔은 대하와 매우 유사하여 구분이 다소 어렵지만 이마뿔(액각)이 대하에 ...,etc,흰다리새우의 형태적 특징은 흰다리새우의 생물학적 특성 부분에서 자세히 설명되어 있습...,흰다리새우의 형태적 특징은 흰다리새우의 생물학적 특성 부분에서 자세히 설명되어 있습...
70,흰다리새우,성장 적수온이 몇 도야?,23~30°C,etc,성장 적수온은 25~30도입니다.\n\n\n\n\n,성장 적수온은 25~30도입니다.\n\n\n\n\n


In [44]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

# nltk 다운로드 (처음 실행 시 필요)
nltk.download('punkt')

df_selected = df_question[['answer', 'optimized_pred']]


# BLEU 점수 계산
df_selected['BLEU'] = df_selected.apply(
    lambda row: sentence_bleu([row['answer'].split()], row['optimized_pred'].split()), axis=1
)

# BLEU 점수 평균
bleu_avg = df_selected['BLEU'].mean()
print("Average BLEU score:", bleu_avg)


Average BLEU score: 0.056891066538633815


[nltk_data] Downloading package punkt to /home/jiyoon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jiyoon/miniconda3/envs/financial_info/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 co